# deepcars 

In [86]:
import os
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from keras import *
from keras.models import Sequential, Model
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.layers import Activation
from keras.layers.pooling import MaxPooling2D
from keras.layers.convolutional import Convolution2D
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.preprocessing.image import ImageDataGenerator

In [47]:
DATA_PATH = os.getcwd() +'/5_tensorflow_traffic_light_images/'
TRAIN_PATH = DATA_PATH + 'train/'
VALID_PATH = DATA_PATH + 'valid/'
SAMPLE_PATH = DATA_PATH + 'sample/'

## Separate images

In [48]:
#Create directories
%cd $DATA_PATH
%mkdir -p valid
%mkdir -p sample
%mkdir -p sample/train
%mkdir -p sample/test
%mkdir -p sample/valid

/home/ai/training/deepcars/5_tensorflow_traffic_light_images


In [46]:
%cd ..
%pwd

/home/ai/training/deepcars


u'/home/ai/training/deepcars'

Create a validation set

In [49]:
%cd "$TRAIN_PATH"

/home/ai/training/deepcars/5_tensorflow_traffic_light_images/train


In [75]:
VALID_RATIO = 0.1
sub_dirs = [x for x in os.walk(DATA_PATH+'train/') if x[0] is not DATA_PATH]
for sub_dir in sub_dirs[1:4]:
    
    n_files = len(sub_dir[2])
    folder_name = sub_dir[0].split('/')[-1]
    
    print sub_dir[0]
    print folder_name, n_files
    
    g = glob(sub_dir[0]+'/*.jpg')
    shuf = np.random.permutation(g)
    for i in range(n_files/10):
        file_name = shuf[i].split('/')[-1]
        os.rename(shuf[i], VALID_PATH+folder_name+'/'+file_name)

/home/ai/training/deepcars/5_tensorflow_traffic_light_images/train/green
green 483
/home/ai/training/deepcars/5_tensorflow_traffic_light_images/train/red
red 814
/home/ai/training/deepcars/5_tensorflow_traffic_light_images/train/yellow
yellow 40


## Analyze Images

In [78]:
n_images = 0
img_classes = [x for x in os.walk(TRAIN_PATH) if x[0] is not TRAIN_PATH]
for img_class in img_classes:
    n_images += len(img_class[2])
    print img_class[0], len(img_class[2])

print "Total images: ", n_images

/home/ai/training/deepcars/5_tensorflow_traffic_light_images/train/green 435
/home/ai/training/deepcars/5_tensorflow_traffic_light_images/train/red 733
/home/ai/training/deepcars/5_tensorflow_traffic_light_images/train/yellow 36
Total images:  1204


### Resolution

In [ ]:
from PIL import Image
import numpy as np

image_resolutions =[]

for img_class in img_classes:
    for filepath in img_class[2]:
        try:
            full_path = os.path.join(img_class[0], filepath)
#             print full_path
            with Image.open(full_path) as im:
                image_resolutions.append(im.size)
        except:
#             print 'error'
            pass

image_resolutions = np.asarray(image_resolutions)
image_resolutions.mean(axis=0)

## Histogram of image resolutions

In [ ]:
import matplotlib.pyplot as plt
rng = np.random.RandomState(10)  # deterministic random data
# a = np.hstack((rng.normal(size=1000),
#               rng.normal(loc=5, scale=2, size=1000)))
plt.hist(image_resolutions, bins='auto')  # plt.hist passes it's arguments to np.histogram
plt.title("Histogram with 'auto' bins")
plt.show()

Let's pick 20 x 64

In [79]:
input_width, input_height = 20, 64
input_resolution = (input_width, input_height)

# The model

### Data preprocessing

In [80]:
def get_batches(dirname, gen=ImageDataGenerator(), shuffle=True, 
                batch_size=4, class_mode='categorical',
                target_size=input_resolution):
    
    return gen.flow_from_directory(dirname, target_size=target_size,
            class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

In [81]:
def get_data(path, target_size=input_resolution):
    
    batches = get_batches(path, shuffle=False, 
                          batch_size=1, class_mode=None, target_size=target_size)
    return np.concatenate([batches.next() for i in range(batches.nb_sample)])

In [96]:
train_batches = get_batches(TRAIN_PATH)
train_data = get_data(TRAIN_PATH)
valid_batches = get_batches(VALID_PATH)
valid_data = get_data(VALID_PATH)

Found 1204 images belonging to 3 classes.
Found 1204 images belonging to 3 classes.
Found 133 images belonging to 3 classes.
Found 133 images belonging to 3 classes.


### One Hot Encoding the lables

In [97]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

In [99]:
train_classes = train_batches.classes
train_labels = onehot(train_batches.classes)

valid_classes = valid_batches.classes
valid_labels = onehot(valid_batches.classes)

In [100]:
sum(train_labels)

array([ 435.,  733.,   36.])

In [101]:
model =Sequential([
    Flatten(input_shape = (3, 20, 64)),
    Dense(16, activation='relu'),
    Dense(3, activation='softmax')
])

In [111]:
model = Sequential()
model.add(Flatten(input_shape=(3, 20, 64)))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [108]:
model.compile(optimizer=RMSprop(lr=0.1), loss='categorical_crossentropy', metrics=['accuracy'])

In [109]:
model.fit(train_data, train_labels, nb_epoch=3, 
          batch_size=100, validation_data=(valid_data, valid_labels))


Train on 1204 samples, validate on 133 samples
Epoch 1/3
1204/1204 [==============================] - 0s - loss: 10.2947 - acc: 0.3613 - val_loss: 10.3010 - val_acc: 0.3609 400/1204 [========>.....................] - ETA: 0s - loss: 10.3962 - acc: 0.3550
Epoch 2/3
1204/1204 [==============================] - 0s - loss: 10.2947 - acc: 0.3613 - val_loss: 10.3010 - val_acc: 0.3609

In [110]:
sum(model.predict(train_data[:100]))


array([  1.00000000e+02,   8.27177378e-24,   0.00000000e+00], dtype=float32)

In [137]:
%qtconsole

In [152]:
model2 = Sequential([
            Convolution2D(3, 3, 3, input_shape=(3, input_width, input_height)),
            Activation('relu'),
            MaxPooling2D(pool_size=(2, 2)),
            Flatten(),
            Dense(64),
            Activation('relu'),
            Dropout(0.5),
            Dense(3, activation='softmax')
            ])

In [153]:
model2.compile(optimizer=RMSprop(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [154]:
model2.fit(train_data, train_labels, nb_epoch=3, validation_data=(valid_data, valid_labels))

Train on 1204 samples, validate on 133 samples
Epoch 1/3
1204/1204 [==============================] - 0s - loss: 10.4420 - acc: 0.3522 - val_loss: 10.3010 - val_acc: 0.3609 128/1204 [==>...........................] - ETA: 0s - loss: 11.4590 - acc: 0.2891

In [155]:
model2.predict(train_data)

array([[ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.]], dtype=float32)

In [156]:
sum(model2.predict(train_data))

array([ 1204.,     0.,     0.], dtype=float32)